In [3]:
import wandb
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
import warnings

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
)
from datasets import load_dataset
import logging


from sklearn.model_selection import train_test_split
import pandas as pd
import time
import matplotlib.pyplot as plt
import sys, os
import torch
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import wandb

wandb.login()




# Suppress the specific FutureWarning from Hugging Face Transformers
warnings.filterwarnings(
    "ignore",
    message="`tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`",
    category=FutureWarning,
    module="transformers.trainer" # More specific to avoid suppressing other warnings
)
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)


wandb: Currently logged in as: timian (timian-vegg) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [6]:
num_labels = 3
output_dir = f"sentiment_model-{time.strftime("%Y%m%d-%H%M%S")}"
#os.makedirs(output_dir, exist_ok=True)

In [ ]:
sweep_config = {
    'method': 'random',  # random, grid, or bayes
    'metric': {
        'name': 'eval_f1',  # Or 'eval_accuracy', 'eval_loss'
        'goal': 'maximize'  # maximize or minimize
    },
    'parameters': {
        'learning_rate': {
            'distribution': 'log_uniform_values',  # Use log scale for finer search
            'min': 1.0e-6,
            'max': 5.0e-5
        },
        'per_device_train_batch_size': {
            'values': [8, 16]
        },
        'weight_decay': {
            'distribution': 'uniform',
            'min': 0.0,
            'max': 0.1
        },
        'num_train_epochs': {
            'values': [8, 10]
        },
        'early_stopping_patience': {
            'values': [3, 5]
        },
        # New parameters to help prevent overfitting:
        'dropout_rate': {
            'values': [0.1, 0.2, 0.3]  # Adjust based on your model’s support for dropout
        },
        'label_smoothing': {
            'values': [0.0, 0.1, 0.2]  # 0.0 means no smoothing
        },
        # Optionally, if your training code supports it:
        'max_grad_norm': {
            'distribution': 'uniform',
            'min': 0.5,
            'max': 2.0
        }
    }
}



In [8]:
df = pd.read_csv("financial_phrasebank.csv", usecols=["norwegian_sentence", "label"]).rename(columns={"norwegian_sentence": "sentence"})

# Check label distribution
print(df["label"].value_counts())

# Split into train (70%), temp (30%) -> then split temp into validation/test (50% each)
train_ds, temp = train_test_split(df, test_size=0.3, random_state=42)
val_ds, test_ds = train_test_split(temp, test_size=0.5, random_state=42)

print(f"Train size: {len(train_ds)}, Validation size: {len(val_ds)}, Test size: {len(test_ds)}")

# Save datasets
train_ds.to_csv("train.csv", index=False)
val_ds.to_csv("validation.csv", index=False)
test_ds.to_csv("test.csv", index=False)

label
1    2535
2    1168
0     514
Name: count, dtype: int64
Train size: 2951, Validation size: 633, Test size: 633


In [9]:
tokenizer = AutoTokenizer.from_pretrained("NbAiLab/nb-bert-base")

def tokenize_function(examples):
    return tokenizer(
        examples["sentence"],
        padding="max_length",
        truncation=True,
        max_length=128,
    )

dataset = load_dataset(
    "csv",
    data_files={
        "train": "train.csv",
        "validation": "validation.csv",
        "test": "test.csv",
    },
)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Generating train split: 2951 examples [00:00, 86647.89 examples/s]
Generating validation split: 633 examples [00:00, 94272.43 examples/s]
Generating test split: 633 examples [00:00, 79405.26 examples/s]
Map: 100%|██████████| 633/633 [00:00<00:00, 6184.52 examples/s]


In [15]:

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted"),
    }


def train_model():
    wandb.init(project="nb-bert-sweep")
    config = wandb.config


    model = AutoModelForSequenceClassification.from_pretrained("NbAiLab/nb-bert-base", num_labels=num_labels)
    training_args = TrainingArguments(
        output_dir=f"./results_wandb_sweep/{wandb.run.name}",
        report_to="wandb",
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        per_device_train_batch_size=config.per_device_train_batch_size,
        per_device_eval_batch_size=config.per_device_train_batch_size,
        greater_is_better=(sweep_config['metric']['goal'] == 'maximize'),    # lower is better for loss
        metric_for_best_model=sweep_config['metric']['name'],
        weight_decay=config.weight_decay,                                    # To prevent overfitting, TODO NEEDS TUNING, initially increase by a small amount
        num_train_epochs=config.num_train_epochs,                            # Use value from wandb.config
        learning_rate=config.learning_rate,                                  # Very common starting point for BERT fine-tuning, TODO: try 1e-5, 2e-5, 3e-5, 5e-5, or a linear/cosine scheduler
        logging_dir="./logs",
        logging_steps=100,
        save_total_limit=1,
    )

    early_stopping_callback = EarlyStoppingCallback(
        early_stopping_patience=config.early_stopping_patience,
        early_stopping_threshold=0.001 # A small threshold for improvement
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[early_stopping_callback],
    )

    trainer.train()
    wandb.finish()


In [16]:
sweep_id = wandb.sweep(sweep=sweep_config, project="nb-bert-sweep")

wandb.agent(sweep_id, function=train_model, count=10)
#last id: 1ehfsdt4

Create sweep with ID: 1ehfsdt4
Sweep URL: https://wandb.ai/timian-vegg/nb-bert-sweep/sweeps/1ehfsdt4


wandb: Agent Starting Run: 62xsc0tv with config:
wandb: 	early_stopping_patience: 3
wandb: 	learning_rate: 2.0916934518335293e-05
wandb: 	num_train_epochs: 10
wandb: 	per_device_train_batch_size: 8
wandb: 	weight_decay: 0.08448659789034704


The following layers were not sharded: bert.encoder.layer.*.attention.self.query.bias, bert.embeddings.position_embeddings.weight, classifier.bias, bert.encoder.layer.*.attention.self.key.weight, bert.encoder.layer.*.intermediate.dense.weight, bert.encoder.layer.*.attention.output.dense.weight, bert.encoder.layer.*.intermediate.dense.bias, bert.pooler.dense.weight, bert.pooler.dense.bias, bert.encoder.layer.*.attention.output.LayerNorm.bias, bert.embeddings.token_type_embeddings.weight, bert.encoder.layer.*.attention.self.query.weight, bert.encoder.layer.*.output.LayerNorm.weight, bert.encoder.layer.*.output.dense.weight, bert.encoder.layer.*.attention.self.value.weight, bert.encoder.layer.*.output.dense.bias, bert.encoder.layer.*.attention.self.key.bias, bert.encoder.layer.*.attention.output.LayerNorm.weight, bert.encoder.layer.*.output.LayerNorm.bias, bert.embeddings.LayerNorm.bias, bert.encoder.layer.*.attention.output.dense.bias, bert.encoder.layer.*.attention.self.value.bias, bert

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.452500,0.521550,0.857820,0.859314
2,0.246600,0.436199,0.895735,0.895646
3,0.181000,0.576024,0.868878,0.870877
4,0.086200,0.642492,0.883096,0.883211
5,0.068400,0.538513,0.903633,0.903680
6,0.027500,0.591398,0.903633,0.904139
7,0.030700,0.624302,0.903633,0.903707
8,0.006800,0.621679,0.902054,0.902011


eval/accuracy,▁▇▃▅████
eval/f1,▁▇▃▅████
eval/loss,▄▁▆█▄▆▇▇
eval/runtime,▁▂▃▂▂▂▂█
eval/samples_per_second,█▇▆▆▇▇▇▁
eval/steps_per_second,█▇▆▆▇▇▇▁
train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/grad_norm,▁▁▂▂▂▃▂▂▁▁▁▁▁▁▁▁▃▁▁▁▁█▁▁▁▁▁▁▁
train/learning_rate,███▇▇▇▆▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁
train/loss,█▆▅▄▄▄▃▃▂▃▃▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁


...

eval/accuracy,▁▆▅▅▇████
eval/f1,▁▆▅▅▇████
eval/loss,▂▁▃▆▅▆▇█▇
eval/runtime,▁▇▂▁▃▅▄█▆
eval/samples_per_second,█▂▇█▆▄▅▁▃
eval/steps_per_second,█▂▇█▆▄▅▁▃
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/grad_norm,▂▂▄▃▁▅▂█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,███▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁
train/loss,█▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: w2s37izd with config:
wandb: 	early_stopping_patience: 5
wandb: 	learning_rate: 8.939508600928615e-06
wandb: 	num_train_epochs: 8
wandb: 	per_device_train_batch_size: 16
wandb: 	weight_decay: 0.029069016856393005


The following layers were not sharded: bert.encoder.layer.*.attention.self.query.bias, bert.embeddings.position_embeddings.weight, classifier.bias, bert.encoder.layer.*.attention.self.key.weight, bert.encoder.layer.*.intermediate.dense.weight, bert.encoder.layer.*.attention.output.dense.weight, bert.encoder.layer.*.intermediate.dense.bias, bert.pooler.dense.weight, bert.pooler.dense.bias, bert.encoder.layer.*.attention.output.LayerNorm.bias, bert.embeddings.token_type_embeddings.weight, bert.encoder.layer.*.attention.self.query.weight, bert.encoder.layer.*.output.LayerNorm.weight, bert.encoder.layer.*.output.dense.weight, bert.encoder.layer.*.attention.self.value.weight, bert.encoder.layer.*.output.dense.bias, bert.encoder.layer.*.attention.self.key.bias, bert.encoder.layer.*.attention.output.LayerNorm.weight, bert.encoder.layer.*.output.LayerNorm.bias, bert.embeddings.LayerNorm.bias, bert.encoder.layer.*.attention.output.dense.bias, bert.encoder.layer.*.attention.self.value.bias, bert

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.842300,0.561834,0.775671,0.732604
2,0.430100,0.322853,0.876777,0.876549
3,0.178800,0.399453,0.872038,0.872828
4,0.122700,0.458142,0.886256,0.887201
5,0.079900,0.474937,0.890995,0.891274
6,0.051700,0.516741,0.892575,0.892743
7,0.030700,0.540677,0.897314,0.897529
8,0.017900,0.546254,0.895735,0.895876


eval/accuracy,▁▇▇▇████
eval/f1,▁▇▇█████
eval/loss,█▁▃▅▅▇▇█
eval/runtime,▁▁▁▁█▃▆▄
eval/samples_per_second,████▁▆▃▅
eval/steps_per_second,████▁▆▃▅
train/epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇███
train/global_step,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇███
train/grad_norm,▂▂▄▃▃▁▁█▁▁▅▅▁▁
train/learning_rate,█▇▇▆▆▅▅▄▄▃▃▂▂▁
train/loss,█▆▄▃▂▂▂▂▂▁▁▁▁▁


In [ ]:

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted"),
    }


def train_model():
    wandb.init(project="nb-bert-sweep")
    config = wandb.config


    model = AutoModelForSequenceClassification.from_pretrained("NbAiLab/nb-bert-base", num_labels=num_labels)
    training_args = TrainingArguments(
        output_dir=f"./results_wandb_sweep/{wandb.run.name}",
        report_to="wandb",
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        per_device_train_batch_size=config.per_device_train_batch_size,
        per_device_eval_batch_size=config.per_device_train_batch_size,
        greater_is_better=(sweep_config['metric']['goal'] == 'maximize'),    # lower is better for loss
        metric_for_best_model=sweep_config['metric']['name'],
        weight_decay=config.weight_decay,                                    # To prevent overfitting, TODO NEEDS TUNING, initially increase by a small amount
        num_train_epochs=config.num_train_epochs,                            # Use value from wandb.config
        learning_rate=config.learning_rate,                                  # Very common starting point for BERT fine-tuning, TODO: try 1e-5, 2e-5, 3e-5, 5e-5, or a linear/cosine scheduler
        logging_dir="./logs",
        logging_steps=100,
        save_total_limit=1,
    )

    early_stopping_callback = EarlyStoppingCallback(
        early_stopping_patience=config.early_stopping_patience,
        early_stopping_threshold=0.001 # A small threshold for improvement
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[early_stopping_callback],
    )

    trainer.train()
    wandb.finish()
